# Loading dataset

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# Считываем базу знаний
knowledge_base = pd.read_excel('/kaggle/input/faq-vseros/01__.xlsx')
knowledge_base.rename(columns={'Вопрос из БЗ': 'question', 'Ответ из БЗ': 'answer'}, inplace=True)

# Считываем реальные кейсы использования
real_cases = pd.read_excel('/kaggle/input/faq-vseros/02__.xlsx')
real_cases.rename(columns={'Вопрос пользователя': 'question', 'Ответ сотрудника': 'answer'}, inplace=True)

# Считываем эталонные вопросы-ответы из реальных кейсов
neighbors_best_faq = real_cases[['Вопрос из БЗ', 'Ответ из БЗ', 'Классификатор 1 уровня', 'Классификатор 2 уровня']]
neighbors_best_faq.rename(columns={'Вопрос из БЗ': 'question', 'Ответ из БЗ': 'answer'}, inplace=True)

data = pd.concat([
    knowledge_base[['question', 'answer', 'Классификатор 1 уровня', 'Классификатор 2 уровня']],
    real_cases[['question', 'answer', 'Классификатор 1 уровня', 'Классификатор 2 уровня']],
    neighbors_best_faq[['question', 'answer', 'Классификатор 1 уровня', 'Классификатор 2 уровня']],
])

data.head(5)

,question,answer,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
2,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


## Разобьем выборку на train/val

In [2]:
# Создание простых переходов от айдишника к лэйблу и обратно
id2label = data['Классификатор 1 уровня'].value_counts().to_dict()
label2id = {k: i for i, k in enumerate(id2label)}
id2label = {i: k for i, k in enumerate(id2label)}

In [3]:
from sklearn.model_selection import train_test_split

texts = [row['question'] for _, row in data.iterrows()]
labels = [label2id[row['Классификатор 1 уровня']] for _, row in data.iterrows()]

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

## Создадим токены для датасета

In [4]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

## Определим кастомный датасет

In [6]:
import torch
from torch.utils.data import Dataset

class CustomTextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [7]:
from torch.utils.data import DataLoader

# Создадим датасеты
train_dataset = CustomTextDataset(train_encodings, train_labels)
test_dataset = CustomTextDataset(test_encodings, test_labels)

# Metrics

In [30]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    # Вычисляем метрики для каждого класса
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None)
    
    # Вычисляем средние значения метрик
    macro_f1 = np.mean(f1)
    macro_precision = np.mean(precision)
    macro_recall = np.mean(recall)
    
    # Вычисляем общую точность
    accuracy = accuracy_score(labels, predictions)
    
    # Вычисляем weighted F1-score
    weighted_f1 = precision_recall_fscore_support(labels, predictions, average='weighted')[2]

    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'weighted_f1': weighted_f1
    }


# Train

In [39]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=len(label2id), id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
training_args = TrainingArguments(
    output_dir="my_baseline_classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    disable_tqdm=True,
    report_to=[],
    metric_for_best_model="weighted_f1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

{'eval_loss': 2.0279428958892822, 'eval_accuracy': 0.25906735751295334, 'eval_macro_f1': 0.04115226337448559, 'eval_macro_precision': 0.025906735751295335, 'eval_macro_recall': 0.1, 'eval_weighted_f1': 0.10661208128105076, 'eval_runtime': 2.5705, 'eval_samples_per_second': 150.164, 'eval_steps_per_second': 5.057, 'epoch': 1.0}
{'eval_loss': 1.8030604124069214, 'eval_accuracy': 0.4067357512953368, 'eval_macro_f1': 0.16556829393027855, 'eval_macro_precision': 0.1466011862108731, 'eval_macro_recall': 0.21021782799748903, 'eval_weighted_f1': 0.29616796969730547, 'eval_runtime': 2.4922, 'eval_samples_per_second': 154.885, 'eval_steps_per_second': 5.216, 'epoch': 2.0}
{'eval_loss': 1.4868924617767334, 'eval_accuracy': 0.5155440414507773, 'eval_macro_f1': 0.2660047771233925, 'eval_macro_precision': 0.2510795941279444, 'eval_macro_recall': 0.30383989073819584, 'eval_weighted_f1': 0.41506377205626244, 'eval_runtime': 2.4752, 'eval_samples_per_second': 155.945, 'eval_steps_per_second': 5.252, 'e

TrainOutput(global_step=980, training_loss=0.7604490941884566, metrics={'train_runtime': 603.7, 'train_samples_per_second': 51.085, 'train_steps_per_second': 1.623, 'train_loss': 0.7604490941884566, 'epoch': 20.0})

In [41]:
eval_results = trainer.evaluate()
eval_results

{'eval_loss': 0.6737048029899597, 'eval_accuracy': 0.8264248704663213, 'eval_macro_f1': 0.7872708018722336, 'eval_macro_precision': 0.8300291795844442, 'eval_macro_recall': 0.7883692646332385, 'eval_weighted_f1': 0.8247022611015289, 'eval_runtime': 2.5246, 'eval_samples_per_second': 152.895, 'eval_steps_per_second': 5.149, 'epoch': 20.0}


{'eval_loss': 0.6737048029899597,
 'eval_accuracy': 0.8264248704663213,
 'eval_macro_f1': 0.7872708018722336,
 'eval_macro_precision': 0.8300291795844442,
 'eval_macro_recall': 0.7883692646332385,
 'eval_weighted_f1': 0.8247022611015289,
 'eval_runtime': 2.5246,
 'eval_samples_per_second': 152.895,
 'eval_steps_per_second': 5.149,
 'epoch': 20.0}

# Inference модели

In [75]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/my_baseline_classifier/checkpoint-980")
inputs = tokenizer('Как узнать версию кринжей?', return_tensors="pt")

In [76]:
import torch

model.eval()
with torch.no_grad():
    outputs = model(**inputs.to(torch.device('cuda')))

In [77]:
predictions = torch.softmax(outputs.logits, dim=1)
predicted_class = torch.argmax(predictions, dim=1).item()
print(predictions)
id2label[predicted_class]

tensor([[0.0668, 0.3779, 0.0417, 0.0159, 0.0122, 0.0086, 0.0291, 0.2255, 0.0803,
         0.1165, 0.0256]], device='cuda:0')


'ВИДЕО'